In [2]:
# to register
from pt.modeling.meta_arch.rcnn import GuassianGeneralizedRCNN
from pt.modeling.proposal_generator.rpn import GuassianRPN
from pt.modeling.roi_heads.roi_heads import GuassianROIHead
import pt.data.datasets.builtin
from pt.modeling.backbone.vgg import build_vgg_backbone
from pt.modeling.anchor_generator import DifferentiableAnchorGenerator


from pt.modeling.meta_arch.ts_ensemble import EnsembleTSModel
from detectron2.checkpoint import DetectionCheckpointer
import torch


/home/superorange5/.local/lib/python3.8/site-packages/torchvision/transforms/transforms.py:803: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


## load config

In [3]:
from detectron2.config import get_cfg
from pt import add_config

def setup(config_file):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    add_config(cfg)
    cfg.merge_from_file(config_file)
    #cfg.merge_from_list(args.opts)
    
    #default_setup(cfg, args)
#     cfg.SOLVER.IMG_PER_BATCH_LABEL = 64
#     cfg.SOLVER.IMG_PER_BATCH_UNLABEL = 64

    cfg.freeze()
    return cfg

## load model

In [4]:
from detectron2.modeling import build_model
from pt.engine.trainer import PTrainer
from detectron2.checkpoint import DetectionCheckpointer




def load_TSmodel(cfg_path, model_path):
    cfg = setup(cfg_path)
    #cfg.defrost()
    #cfg.MODEL.WEIGHTS = model_path
    
    Trainer =PTrainer
    model = Trainer.build_model(cfg)
    model_teacher = Trainer.build_model(cfg)
    ensem_ts_model = EnsembleTSModel(model_teacher, model)    
    
    
    DetectionCheckpointer(ensem_ts_model).resume_or_load(model_path, resume=False)
    
    return ensem_ts_model

In [5]:
import os
model_folder = 'keep_experiments'

def get_model(dataset_name,model_num):
    if model_num =='final':
        model_name ='model_final.pth'
    else:
        model_name = 'model_{0:07d}.pth'.format(model_num)
    model_path = os.path.join(model_folder,dataset_name,model_name)
    cfg_path = os.path.join(model_folder,dataset_name,'cfg.yaml')
    print(cfg_path)
    print(model_path)
    return load_TSmodel(cfg_path, model_path)

In [6]:

#---------skf2c-------
model_s2c = get_model('s2c', 15999)
model_k2c = get_model('k2c', 19999)
model_f2c = get_model('f2c', 'final') #8class
model_list_skf2c=[model_s2c,model_k2c,model_f2c]


keep_experiments/s2c/cfg.yaml
keep_experiments/s2c/model_0015999.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
keep_experiments/k2c/cfg.yaml
keep_experiments/k2c/model_0019999.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
keep_experiments/f2c/cfg.yaml
keep_experiments/f2c/model_final.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


In [7]:
#--------ck2bdd-------
model_c2b = get_model('c2b', 11999)
model_k2b = get_model('k2b', 'final')
model_list_ck2b=[model_c2b,model_k2b]



keep_experiments/c2b/cfg.yaml
keep_experiments/c2b/model_0011999.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------
keep_experiments/k2b/cfg.yaml
keep_experiments/k2b/model_final.pth
-------- pretrained model loaded ---------
-------- pretrained model loaded ---------


## evaluation (defaults.test)

In [10]:
model_c2b.modelStudent.state_dict()['proposal_generator.rpn_head.anchor_deltas.bias']

tensor([ 4.9567e-03,  4.1854e-02, -8.4723e-01, -5.3333e-01, -1.7204e+00,
        -1.9618e+00, -1.1261e+00, -1.2538e+00,  7.7744e-03,  2.2535e-02,
        -2.2750e-01, -4.8190e-01, -7.0024e-01, -8.2286e-01, -5.7922e-01,
        -6.0722e-01, -3.5713e-02,  6.8824e-02, -2.9323e-01, -1.2816e+00,
        -9.5808e-01, -1.4632e+00, -6.4602e-01, -8.0602e-01, -1.1137e-03,
         4.0937e-03, -1.2114e-01, -3.2870e-02, -2.6557e-01, -2.5672e-01,
        -2.0734e-01, -1.9663e-01,  4.5403e-03, -7.9842e-03, -4.0569e-03,
        -1.0426e-01, -3.1593e-01, -3.0426e-01, -2.6025e-01, -2.3013e-01,
        -7.1750e-03,  1.7197e-03, -2.0883e-02, -4.9887e-02, -1.4983e-01,
        -1.3809e-01, -1.3263e-01, -1.1133e-01,  3.7207e-03,  9.6099e-04,
        -4.0151e-02, -2.8066e-02, -9.2451e-02, -9.5428e-02, -8.6735e-02,
        -7.9944e-02,  5.4622e-03,  4.5548e-03, -4.3455e-02, -6.1940e-02,
        -1.1308e-01, -1.0045e-01, -1.0183e-01, -8.1233e-02,  2.0255e-03,
         4.3722e-03, -2.2436e-02, -2.0772e-02, -5.6

In [14]:
#results = OrderedDict()
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import PascalVOCDetectionEvaluator
from detectron2.evaluation import inference_on_dataset

cfg = setup("keep_experiments/c2b/cfg.yaml")

dataset_name = 'VOC2007_bddvalsmall'
test_data_loader = build_detection_test_loader(cfg, dataset_name)
evaluator = PascalVOCDetectionEvaluator(dataset_name)




In [15]:
data_loader_iter = iter(test_data_loader)
data = data_loader_iter.next()

In [16]:
data

[{'file_name': 'data/VOC2007_bddval/JPEGImages/b1c66a42-6f7d68ca.jpg',
  'image_id': 'b1c66a42-6f7d68ca',
  'height': 720,
  'width': 1280,
  'image': tensor([[[254, 255, 255,  ...,  57,  54,  51],
           [243, 250, 253,  ...,  56,  53,  50],
           [206, 231, 243,  ...,  54,  51,  48],
           ...,
           [ 49,  49,  49,  ...,  28,  28,  28],
           [ 47,  47,  47,  ...,  26,  26,  26],
           [ 47,  47,  47,  ...,  25,  25,  25]],
  
          [[254, 255, 254,  ...,  89,  86,  83],
           [241, 249, 251,  ...,  88,  85,  82],
           [202, 227, 239,  ...,  86,  83,  80],
           ...,
           [ 58,  58,  58,  ...,  23,  23,  23],
           [ 56,  56,  56,  ...,  21,  21,  21],
           [ 56,  56,  56,  ...,  20,  20,  20]],
  
          [[222, 219, 216,  ..., 124, 121, 118],
           [207, 215, 217,  ..., 123, 120, 117],
           [174, 199, 211,  ..., 121, 118, 115],
           ...,
           [ 61,  61,  61,  ...,  20,  20,  20],
           

In [46]:
#model_c2b.eval()
results_i = inference_on_dataset(model_c2b.modelStudent, test_data_loader, evaluator)


In [47]:
results_i

OrderedDict([('bbox',
              {'AP': 34.43792240729627,
               'AP50': 55.33559113300493,
               'AP75': 42.69925476822028})])

In [18]:
model = model_c2b.modelStudent

In [29]:


with torch.no_grad():
    output_result=[]
    for model in model_list_ck2b:
        modelStudent= model.modelStudent
        modelStudent.eval()
        output=modelStudent(data)
        output_result.append(output)
    


In [46]:
output_result[1][0]['instances'].scores


tensor([0.9782, 0.9770, 0.9530, 0.9435, 0.9127, 0.9055, 0.9023, 0.9006, 0.8991,
        0.8486, 0.7825, 0.7520, 0.3332, 0.2876, 0.2741, 0.2580, 0.2209, 0.2179,
        0.1990, 0.1814, 0.1572, 0.1552, 0.1415, 0.1287, 0.0757, 0.0703, 0.0678,
        0.0633, 0.0492, 0.0355, 0.0342, 0.0307, 0.0280, 0.0252, 0.0211, 0.0204,
        0.0202, 0.0162, 0.0151, 0.0113, 0.0109, 0.0101, 0.0090, 0.0088, 0.0078,
        0.0048, 0.0043, 0.0022, 0.0012], device='cuda:0')

In [23]:
evaluator.reset()
evaluator.process(data, output)


In [25]:
evaluator._predictions

defaultdict(list,
            {0: ['b1c66a42-6f7d68ca 0.971 933.1 339.5 1221.0 481.7',
              'b1c66a42-6f7d68ca 0.971 796.0 363.4 905.5 430.0',
              'b1c66a42-6f7d68ca 0.961 751.1 363.6 820.0 413.1',
              'b1c66a42-6f7d68ca 0.946 39.8 351.0 127.8 400.2',
              'b1c66a42-6f7d68ca 0.931 882.3 378.1 966.8 448.7',
              'b1c66a42-6f7d68ca 0.899 247.1 350.0 349.4 397.7',
              'b1c66a42-6f7d68ca 0.896 599.8 355.0 635.8 385.3',
              'b1c66a42-6f7d68ca 0.877 1.0 341.4 58.4 403.1',
              'b1c66a42-6f7d68ca 0.875 733.7 367.5 771.7 404.4',
              'b1c66a42-6f7d68ca 0.756 684.1 360.5 718.3 391.1',
              'b1c66a42-6f7d68ca 0.753 1.0 335.7 10.8 391.8',
              'b1c66a42-6f7d68ca 0.628 204.9 344.2 258.1 386.0',
              'b1c66a42-6f7d68ca 0.594 215.8 359.8 300.0 455.8',
              'b1c66a42-6f7d68ca 0.539 709.3 365.5 743.6 398.0',
              'b1c66a42-6f7d68ca 0.457 692.2 362.0 737.2 395.2',
          

In [20]:
output[0]['instances'].image_size

(720, 1280)

In [101]:
output[0]['instances'].pred_boxes
output[0]['instances'].scores_logists
output[0]['instances'].boxes_sigma
output[0]['instances'].scores

tensor([0.9708, 0.9707, 0.9612, 0.9462, 0.9312, 0.8988, 0.8963, 0.8774, 0.8749,
        0.7561, 0.7531, 0.6275, 0.5939, 0.5390, 0.4565, 0.4178, 0.2566, 0.1998,
        0.1848, 0.1812, 0.1348, 0.1121, 0.0759, 0.0390, 0.0330, 0.0310, 0.0206,
        0.0180, 0.0171, 0.0161, 0.0135, 0.0127, 0.0123, 0.0106, 0.0099, 0.0094],
       device='cuda:0')

In [95]:
output[0]['instances'].get_fields

<bound method Instances.get_fields of Instances(num_instances=36, image_height=720, image_width=1280, fields=[pred_boxes: Boxes(tensor([[9.3210e+02, 3.3852e+02, 1.2210e+03, 4.8170e+02],
        [7.9497e+02, 3.6244e+02, 9.0552e+02, 4.3002e+02],
        [7.5011e+02, 3.6257e+02, 8.2003e+02, 4.1306e+02],
        [3.8800e+01, 3.4999e+02, 1.2780e+02, 4.0018e+02],
        [8.8130e+02, 3.7710e+02, 9.6676e+02, 4.4869e+02],
        [2.4614e+02, 3.4897e+02, 3.4941e+02, 3.9769e+02],
        [5.9876e+02, 3.5402e+02, 6.3575e+02, 3.8532e+02],
        [0.0000e+00, 3.4040e+02, 5.8351e+01, 4.0309e+02],
        [7.3273e+02, 3.6651e+02, 7.7169e+02, 4.0439e+02],
        [6.8310e+02, 3.5948e+02, 7.1826e+02, 3.9106e+02],
        [0.0000e+00, 3.3469e+02, 1.0768e+01, 3.9184e+02],
        [2.0393e+02, 3.4324e+02, 2.5814e+02, 3.8604e+02],
        [2.1476e+02, 3.5879e+02, 3.0001e+02, 4.5576e+02],
        [7.0833e+02, 3.6453e+02, 7.4357e+02, 3.9798e+02],
        [6.9122e+02, 3.6096e+02, 7.3717e+02, 3.9521e+02],
  

In [52]:
for idx, inputs in enumerate(test_data_loader):
    output = model_c2b.modelStudent(input)
    print(output)
    

TypeError: 'method' object is not subscriptable

In [63]:
print(next(g))

4


In [51]:
def get_all_inputs_outputs(data_loader,model):
    for data in data_loader:
        yield data, model(data)


evaluator.reset()
for inputs, outputs in get_all_inputs_outputs(test_data_loader,model_c2b.modelTeacher):
      evaluator.process(inputs, outputs)
eval_results = evaluator.evaluate()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
for idx, inputs in enumerate(test_data_loader):

    outputs = model_teacher(inputs)
    evaluator.process(inputs, outputs)
results = evaluator.evaluate()